<div style="position: relative;">
<img src="https://user-images.githubusercontent.com/7065401/98728503-5ab82f80-2378-11eb-9c79-adeb308fc647.png"></img>

<h1 style="color: white; position: absolute; top:27%; left:10%;">
     INE Bootcamp
</h1>
<h2 style="color: white; position: absolute; top:36%; left:10%;">
    Data Analysis, Visualization and Predictive Modeling
</h2> 

<h3 style="color: #ef7d22; font-weight: normal; position: absolute; top:58%; left:10%;">
    <b>David Mertz, Ph.D.</b>
</h3>

<h3 style="color: #ef7d22; font-weight: normal; position: absolute; top:63%; left:10%;">
    <b>Data Scientist</b>
</h3>
</div>

<div style="width: 100%; height: 200px; background-color: #222; text-align: center; padding-top: 20px; margin-bottom: 40px;">
<br><br>

<h1 style="color: white; font-weight: bold;">
    Advanced Pandas: Timeseries and Groupby
</h1>

<br><br> 
</div>

<img src="https://user-images.githubusercontent.com/7065401/75165824-badf4680-5701-11ea-9c5b-5475b0a33abf.png" style="width:200px; float: right; margin: 0 40px 40px 40px;"/>

The Pandas (Panel Data) Python library is a very powerful tool for data manipulation and analysis.  We will talk about it throughout several lessons of this bootcamp, and even assume familiarity with Pandas in later lessons. Here we look at some more advanced capabilities.

There are **a lot** of capabilities within Pandas.  This short bootcamp cannot touch upon every capability within it.  But for this lesson, we want to touch on special handling of datetime data and using grouping playing around with some hypothetical data on users who have accounts at some service.


In [ ]:
import pandas as pd
# Read in from efficient parquet format
accounts = pd.read_parquet('data/accounts.parq')
accounts

<h2 style="font-weight: bold;">
    Categories
</h2>

![orange-divider](https://user-images.githubusercontent.com/7065401/98619088-44ab6000-22e1-11eb-8f6d-5532e68ab274.png)

Much data in the world is *categorical*; that is, it takes on one of a comparatively few number of distinct values (often represented as strings).  Some categories might be more common, which is often important to notice. For example, favorite flower seems to break down pretty equally among the options.

In [ ]:
accounts.Favorite_Flower.value_counts()

In contrast, some names are much more common than others.

In [ ]:
accounts.Name.value_counts()

<h2 style="font-weight: bold;">
    Date, time, and datetime
</h2>

![orange-divider](https://user-images.githubusercontent.com/7065401/98619088-44ab6000-22e1-11eb-8f6d-5532e68ab274.png)

An important kind of data is timestamp or datetime information.  In our dataset, we have two fields in Pandas' special `datetime64[ns]` datatype, which as the name suggests can store precision down to nanoseconds.  It can also be inspected in interesting and flexible ways.

In [ ]:
accounts.dtypes

The special `.dt` accessor pokes into datetime columns.

In [ ]:
accounts.Account_Closed.dt.year

It looks like some accounts have long been closed, and a few are paid long in advance.  But most of them are expiring over the previous or next few years.

In [ ]:
(accounts
     .Account_Closed
     .dt
     .year
     .value_counts()
     .sort_index()
     .plot(title="Account closing years"));

We can pull off a varity of elements of datetimes to work with.

In [ ]:
# Only the date without the time
accounts.Account_Opened.dt.date.head()

In [ ]:
# Only the time without the date
accounts.Account_Opened.dt.time.head()

In [ ]:
# Only the seconds
accounts.Account_Opened.dt.second.head()

<h2 style="font-weight: bold;">
    Time deltas
</h2>

![orange-divider](https://user-images.githubusercontent.com/7065401/98619088-44ab6000-22e1-11eb-8f6d-5532e68ab274.png)

Arithmetic on datetime objects is powerful and relatively intuitive.  For example, what was the duration for which each account was held?

In [ ]:
accounts.Account_Closed - accounts.Account_Opened

We can aggregate timedeltas in much the way we do plain numbers.

In [ ]:
(accounts.Account_Closed - accounts.Account_Opened).mean()

In [ ]:
(accounts.Account_Closed - accounts.Account_Opened).min()

In [ ]:
(accounts.Account_Closed - accounts.Account_Opened).max()

Perhaps in an advertising promotion, we will add 60 days until account expiration for all accounts that are currently active.  Let us look at some upcoming expirations first:

In [ ]:
(accounts[accounts.Account_Closed > pd.Timestamp.now()]
     .sort_values('Account_Closed')
     .head()
)

Let us add the extra enrollment.

In [ ]:
accounts.loc[accounts.Account_Closed > pd.Timestamp.now(), 
                'Account_Closed'] += pd.Timedelta(days=60)

In [ ]:
(accounts[accounts.Account_Closed > pd.Timestamp.now()]
     .sort_values('Account_Closed')
     .head()
)

<h2 style="font-weight: bold;">
    Grouping data
</h2>

![orange-divider](https://user-images.githubusercontent.com/7065401/98619088-44ab6000-22e1-11eb-8f6d-5532e68ab274.png)

Commonly, we are interested in aggregations of rows sharing a common categorical value.  For example, what is the average age and account balance of members who like yellow and lilies?

In [ ]:
mask = (accounts.Favorite_Color == "Yellow") & (accounts.Favorite_Flower == 'Lily')
accounts[mask].Age.median()

In [ ]:
accounts[mask].Balance.mean()

Very often we would like to generalize this kind of question though, rather than filtering with a specific mask.  That's where the `.groupby()` method comes in.

In [ ]:
accounts.groupby(['Favorite_Color', 'Favorite_Flower']).mean()

In those results, age seems to vary merely randomly, but balance looks more systematic.  Let us explore that.

In [ ]:
accounts.groupby('Favorite_Flower').Balance.mean().sort_values()

Those differences do not jump out at us, but more nuanced statistical tests are certainly possible. Let us try a different grouping.

In [ ]:
accounts.groupby('Favorite_Color').Balance.mean().sort_values()

Here we see a pretty unmistakeable pattern.  Members who like yellow have a notably high balance; those who like green have a notably low balance.  Black looks to be slightly outside the trend, but it is harder to be certain if it could be random variation.

We might want to look at several different aggregations at the same time to understand general properties of category class distribution.  For example, are members with different names of characteristically different ages?

In [ ]:
accounts.groupby('Name').Age.agg(['min', 'max', 'median', 'std', 'count'])

<div style="width: 100%; height: 200px; background-color: #ef7d22; text-align: center; padding-top: 20px; margin-bottom: 40px;">
<br><br>

<h1 style="color: white; font-weight: bold;">
    Exercises
</h1>

<br><br> 
</div>

<h2 style="font-weight: bold;">
    Look for pending events
</h2>

![orange-divider](https://user-images.githubusercontent.com/7065401/98619088-44ab6000-22e1-11eb-8f6d-5532e68ab274.png)

The membership department would like to send membership extension offers to all the over-65 years-old members who have accounts closed in the last six weeks, or that will close in the future.  They wish to prioritize their mailings by the account balance of each qualifying member (from highest to lowest balance).  Produce a DataFrame representing their desired information.

In [ ]:
# your code here...

<h2 style="font-weight: bold;">
    Grouping by month
</h2>

![orange-divider](https://user-images.githubusercontent.com/7065401/98619088-44ab6000-22e1-11eb-8f6d-5532e68ab274.png)

You will need to think a bit about combining things you have learned already to do this exercise.  We would like to know whether users who joined in different months tend to have different attributes.  Write a few lines of Pandas code to answer that question.  Consider that we have both numeric data for balance and age, and also categorical data for favorite flower and color. For categoricals, we might ask what is most common, for example.

In [ ]:
# your code here...

<div style="width: 100%; height: 400px; background-color: #222; text-align: center; padding-top: 120px;">
<br><br>

<h1 style="color: white; font-weight: bold;">
    Review and questions
</h1>

<br><br> 
</div>

---
<div style="position: relative; text-align: right;">
<img src="https://user-images.githubusercontent.com/7065401/98614301-dcf01780-22d6-11eb-9c8f-65ebfceac6f6.png" style="width: 130px; display: inline-block;"></img>

<img src="https://user-images.githubusercontent.com/7065401/98864025-08deda80-2448-11eb-9600-22aa17884cdf.png" style="height: 100%; max-height: inherit; position: absolute; top: 20%; left: 0px;"></img>
<br>

<h2 style="font-weight: bold;">
    David Mertz, Ph.D.
</h2>

<h3 style="color: #ef7d22; margin-top: 0.8em">
    Data Scientist
</h3>
<hr>
<br><br>

<p style="font-size: 80%; text-align: right; margin: 10px 0px;">
    david.mertz@gmail.com
</p>
<p style="font-size: 80%; text-align: right; margin: 10px 0px;">
    linkedin.com/in/dmertz/
</p>

</div>

<br><br><br>